### Imports

In [1]:
import pandas as pd
from auto_ts import auto_timeseries
import dill
import talib
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

Imported auto_timeseries version:0.0.90. Call by using:
model = auto_timeseries(score_type='rmse',
        time_interval='M', non_seasonal_pdq=None, seasonality=False,
        seasonal_period=12, model_type=['best'], verbose=2, dask_xgboost_flag=0)
model.fit(traindata, ts_column,target)
model.predict(testdata, model='best')



In [2]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Dataset

In [3]:
columns = [
    'Open_time',
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]
columns_to_model=[
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    # 'SMA_20',
    # 'EMA_20',
    # 'Upper_Band',
    # 'Middle_Band',
    # 'Lower_Band',
    # 'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

In [4]:
dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])

In [5]:
display(dataset.head())

,Open_time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Sentimiento,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Sentimiento_coin,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Sentimiento_referentes,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
0,2021-09-27,28.84,30.26,27.50,27.71,11094418.10,320340017.82,449178.00,43160.90,44350.00,42098.00,42147.35,39776.84,1728912299.80,1001487.00,3059.36,3164.94,2922.71,2925.59,510130.73,1564096695.37,1043885.00,344.00,357.60,335.00,335.50,956544.07,332372476.51,457187.00,31.85,30.83,38.30,31.85,25.39,44.11,0.02,0.96,21.20,24.79,33.78,-86.30,4.00,29.23,0.00,0.00,0.00,pos,139,61.00,28.00,50.00,17.83,pos,135,71.00,26.00,38.00,30.62,neu,1.00,0.00,0.00,1.00,0.00,81.00,525.00,364.00,270000.00,Lateral
1,2021-09-28,27.72,28.38,26.14,26.31,9784154.53,267560973.69,362304.00,42147.35,42787.38,40888.00,41026.54,43372.26,1816305387.84,1045389.00,2924.76,2970.82,2789.00,2804.91,511325.46,1477983245.67,928494.00,335.50,344.60,330.00,333.00,922077.23,310828343.70,417006.00,31.77,30.40,38.44,31.77,25.11,41.83,-0.33,0.70,21.98,16.10,24.47,-109.46,3.88,19.19,4.00,2.00,5273.00,pos,93,40.00,21.00,32.00,10.18,pos,122,57.00,23.00,42.00,19.89,neu,2.00,1.00,1.00,0.00,0.02,87.00,472.00,331.00,204000.00,Bajista
2,2021-09-29,26.31,28.59,26.11,27.28,10207287.33,280535376.38,376232.00,41025.01,42590.00,40753.88,41524.28,33511.53,1399625476.06,884909.00,2805.15,2950.00,2782.00,2850.45,411305.09,1180681056.16,748804.00,332.90,375.40,331.20,367.70,1696420.04,612547661.88,653011.00,31.65,30.10,38.55,31.65,24.74,43.99,-0.52,0.46,22.61,12.88,17.92,-92.93,3.78,23.82,22.00,40.00,54144.00,pos,112,61.00,19.00,32.00,23.13,pos,145,71.00,20.00,54.00,30.30,neu,0.00,0.00,0.00,0.00,0.00,64.00,594.00,495.00,216000.00,Alcista
3,2021-09-30,27.28,28.99,27.13,28.62,8201551.44,230622998.43,339737.00,41524.29,44141.37,41410.17,43824.10,46381.23,2005486003.56,1197815.00,2850.45,3048.42,2837.19,3000.61,506896.76,1514341937.31,992243.00,367.70,388.80,366.40,387.50,1163674.21,439628574.42,551245.00,31.62,29.96,38.56,31.62,24.67,46.92,-0.56,0.26,22.99,16.42,15.13,-60.03,3.64,28.34,15.00,23.00,39220.00,pos,116,46.00,28.00,42.00,7.68,pos,147,73.00,26.00,48.00,30.90,pos,2.00,1.00,0.00,1.00,0.75,77.00,419.00,464.00,202000.00,Alcista
4,2021-10-01,28.61,32.33,28.50,31.94,13176348.21,402700970.65,735059.00,43820.01,48495.00,43283.03,48141.61,66244.87,3076094886.59,1771237.00,3000.62,3330.00,2969.07,3309.91,648714.62,2063112280.67,1446386.00,387.50,423.40,381.50,421.50,1440336.04,584311192.72,727854.00,31.64,30.15,38.58,31.64,24.69,53.42,-0.31,0.14,21.86,34.90,21.40,34.17,3.65,35.02,24.00,35.00,63183.00,pos,171,72.00,21.00,78.00,30.72,pos,141,70.00,18.00,53.00,30.43,pos,1.00,1.00,0.00,0.00,0.82,71.00,477.00,664.00,492000.00,Alcista


In [6]:
n_days_to_predict = 10

dates = dataset['Open_time'][:-n_days_to_predict]

dataset = dataset.round(2)

feature_dataset = dataset[columns]

validation = feature_dataset[-n_days_to_predict:]
feature_dataset = feature_dataset[:-n_days_to_predict]

In [7]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,2021-09-27,28.84,30.26,27.50,27.71,449178.00,42147.35,39776.84,1001487.00,2925.59,510130.73,1043885.00,335.50,956544.07,457187.00,31.85,30.83,38.30,31.85,25.39,44.11,0.00,0.00,0.00,139,135,1.00,81.00,525.00,364.00,270000.00
1,2021-09-28,27.72,28.38,26.14,26.31,362304.00,41026.54,43372.26,1045389.00,2804.91,511325.46,928494.00,333.00,922077.23,417006.00,31.77,30.40,38.44,31.77,25.11,41.83,4.00,2.00,5273.00,93,122,2.00,87.00,472.00,331.00,204000.00
2,2021-09-29,26.31,28.59,26.11,27.28,376232.00,41524.28,33511.53,884909.00,2850.45,411305.09,748804.00,367.70,1696420.04,653011.00,31.65,30.10,38.55,31.65,24.74,43.99,22.00,40.00,54144.00,112,145,0.00,64.00,594.00,495.00,216000.00
3,2021-09-30,27.28,28.99,27.13,28.62,339737.00,43824.10,46381.23,1197815.00,3000.61,506896.76,992243.00,387.50,1163674.21,551245.00,31.62,29.96,38.56,31.62,24.67,46.92,15.00,23.00,39220.00,116,147,2.00,77.00,419.00,464.00,202000.00
4,2021-10-01,28.61,32.33,28.50,31.94,735059.00,48141.61,66244.87,1771237.00,3309.91,648714.62,1446386.00,421.50,1440336.04,727854.00,31.64,30.15,38.58,31.64,24.69,53.42,24.00,35.00,63183.00,171,141,1.00,71.00,477.00,664.00,492000.00


(946, 31)


,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
946,2024-04-30,6.59,6.67,6.04,6.42,115512.00,60672.00,54947.66,1985671.00,3014.05,561717.49,1292873.00,578.40,766513.45,486465.00,6.93,7.13,7.80,6.93,6.05,34.18,51.00,55.00,48709.00,142,187,1.00,23.00,379.00,377.00,70000.00
947,2024-05-01,6.42,6.94,6.13,6.90,175570.00,58364.97,81166.47,2401089.00,2972.46,624963.78,1365039.00,561.80,669027.32,427425.00,6.85,7.11,7.41,6.85,6.29,43.30,42.00,50.00,83718.00,130,177,0.00,36.00,327.00,340.00,107000.00
948,2024-05-02,6.90,7.41,6.69,7.27,109002.00,59060.61,47583.82,1572898.00,2986.19,365939.72,880167.00,560.50,359794.32,250921.00,6.85,7.12,7.42,6.85,6.29,49.27,87.00,57.00,61208.00,461,374,1.00,25.00,270.00,282.00,71000.00
949,2024-05-03,7.27,7.39,7.00,7.24,109634.00,62882.01,43628.40,1558661.00,3102.61,355825.84,859542.00,587.00,342906.43,257575.00,6.90,7.14,7.44,6.90,6.35,48.86,52.00,40.00,46255.00,573,474,1.00,22.00,386.00,635.00,69000.00
950,2024-05-04,7.24,7.28,7.10,7.12,71120.00,63892.04,24368.69,1113509.00,3117.23,196263.95,575026.00,585.70,197129.25,210303.00,6.91,7.13,7.46,6.91,6.36,46.98,68.00,50.00,34251.00,407,472,0.00,14.00,203.00,232.00,49000.00


(10, 31)

### Models

#### Generacion de los modelos

In [9]:
# Crear un diccionario para almacenar los modelos
trained_models = {}

# Iterar sobre cada columna seleccionada
for column in columns_to_model:
    # Excluir 'Open_time' del bucle de iteración
    if column != 'Open_time':
        # Seleccionar solo la columna actual y la columna de tiempo (Open_time)
        data = feature_dataset[column]
        
        forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 15
             )

        forecaster.fit(y=data)

        trained_models[column] = forecaster

#### Guardo el mejor modelo para cada columna

In [11]:
for column, model in trained_models.items():
    # Guarda el modelo en un archivo usando dill
    with open(f"skforecast/{column}.pkl", "wb") as file:
        dill.dump(model, file)

#### Vuelvo a armar el mejor modelo para cada columna

In [12]:
loaded_models = {}

# Itera sobre los archivos guardados
for column in columns_to_model:
    if column != 'Open_time':
        # Cargar el modelo desde el archivo
        with open(f"skforecast/{column}.pkl", "rb") as file:
            loaded_model = dill.load(file)
        
        # Guardar el modelo cargado en el diccionario
        loaded_models[column] = loaded_model

#### Prediccion N dias en el futuro

In [13]:
def calcular_indicadores_tecnicos(predictions_next_day, feature_dataset, n_days_to_predict):
    # Concatenar la columna Close de predictions_next_day y feature_dataset
    concatenated_df = pd.concat([feature_dataset['Close'], predictions_next_day['Close']], axis=0)

    # Calcular los indicadores sobre el nuevo dataset
    close_prices = np.array(concatenated_df, dtype=np.double)
    sma_20 = talib.SMA(close_prices, timeperiod=30)
    ema_20 = talib.EMA(close_prices, timeperiod=30)
    upper_band, middle_band, lower_band = talib.BBANDS(close_prices, timeperiod=30, nbdevup=2, nbdevdn=2)
    rsi = talib.RSI(close_prices, timeperiod=24)

    # Crear un DataFrame para almacenar los indicadores técnicos
    indicadores = pd.DataFrame({
        'SMA_20': sma_20,
        'EMA_20': ema_20,
        'Upper_Band': upper_band,
        'Middle_Band': middle_band,
        'Lower_Band': lower_band,
        'RSI': rsi
    })

    # Obtener los últimos N registros del dataset con los indicadores
    ultimos_indicadores = indicadores.tail(n_days_to_predict)
    ultimos_indicadores.index = predictions_next_day.index

    # Unir los DataFrames por sus índices
    resultado = pd.concat([predictions_next_day, ultimos_indicadores], axis=1)


    return resultado

In [16]:
def hacer_predicciones_para_N_dias_siguientes(trained_models, n_days_to_predict):
    predictions = {}

    for feature, model in trained_models.items():
        
        # Realizar la predicción para la característica actual
        prediction = model.predict(steps=n_days_to_predict)
        
        # Almacenar la predicción en el diccionario
        predictions[feature] = prediction

    # Convertir el diccionario de predicciones en un DataFrame de pandas
    predictions_df = pd.DataFrame(predictions)

    return predictions_df

In [17]:
predictions_next_day = hacer_predicciones_para_N_dias_siguientes(loaded_models, n_days_to_predict)
display(predictions_next_day)

predictions_next_day = calcular_indicadores_tecnicos(predictions_next_day, feature_dataset, n_days_to_predict )
display(predictions_next_day)

,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
946,6.80,6.92,6.48,6.53,65645.51,64078.69,32023.35,1394760.05,3162.70,332689.93,1176772.63,597.34,433992.24,416444.44,53.25,34.00,36493.43,128.11,110.71,1.00,19.33,251.34,214.94,45250.00
947,6.89,7.06,6.41,6.52,66300.90,64106.61,40369.23,1326045.84,3188.82,376744.85,1049308.48,606.47,545028.16,434097.56,48.06,31.92,33328.81,137.95,120.45,0.78,23.37,295.45,250.55,40300.00
948,7.03,7.17,6.50,6.50,69081.45,64566.34,40032.79,1306721.27,3201.13,393887.17,963653.66,594.53,400342.97,341169.00,44.79,25.72,32004.90,135.25,119.00,0.83,22.00,281.39,256.10,43150.00
949,7.13,7.23,6.63,6.45,90984.48,64912.64,43207.95,1245659.37,3204.52,330653.02,835483.64,584.25,391823.23,317566.14,54.48,28.73,33675.47,149.85,114.82,0.50,25.74,289.56,239.13,46780.00
950,7.16,7.21,6.74,6.41,80009.53,64651.01,26144.95,1250020.39,3194.80,316229.80,938611.48,585.67,361613.62,267757.27,45.10,27.52,32582.48,139.01,111.66,0.81,11.97,284.17,214.20,42310.00
951,7.17,7.17,6.93,6.40,62592.86,64236.66,21591.19,1196746.13,3173.55,307563.31,920489.35,590.06,295617.34,282348.52,38.46,25.04,32816.22,137.59,117.08,1.60,14.69,261.59,228.93,36640.00
952,7.14,7.01,6.89,6.39,80792.92,64352.09,35184.63,1610455.73,3150.45,338416.28,1097848.91,591.16,285045.32,312367.76,48.06,31.95,34949.86,163.30,116.93,0.54,20.12,325.11,257.11,41120.00
953,7.18,6.92,6.85,6.40,90671.52,64129.43,36659.87,1455949.81,3134.99,378833.43,998980.77,591.61,347643.36,426007.57,53.33,30.77,42582.23,153.54,112.85,0.68,21.86,322.16,227.17,45240.00
954,7.20,6.92,6.82,6.40,91023.51,64057.61,36861.15,1338276.42,3178.40,397898.59,1118851.18,588.61,387304.48,475854.14,45.49,28.12,38786.07,170.08,118.32,0.49,24.47,315.07,256.25,40860.00
955,7.20,6.95,6.73,6.41,89472.14,63939.16,37351.19,1405394.00,3189.53,394955.00,978937.03,587.86,406616.90,386083.83,28.19,29.50,33366.09,156.47,110.96,0.38,22.27,302.95,230.01,38570.00


,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI
946,6.80,6.92,6.48,6.53,65645.51,64078.69,32023.35,1394760.05,3162.70,332689.93,1176772.63,597.34,433992.24,416444.44,53.25,34.00,36493.43,128.11,110.71,1.00,19.33,251.34,214.94,45250.00,7.50,7.45,9.27,7.50,5.72,39.22
947,6.89,7.06,6.41,6.52,66300.90,64106.61,40369.23,1326045.84,3188.82,376744.85,1049308.48,606.47,545028.16,434097.56,48.06,31.92,33328.81,137.95,120.45,0.78,23.37,295.45,250.55,40300.00,7.41,7.39,9.11,7.41,5.71,39.19
948,7.03,7.17,6.50,6.50,69081.45,64566.34,40032.79,1306721.27,3201.13,393887.17,963653.66,594.53,400342.97,341169.00,44.79,25.72,32004.90,135.25,119.00,0.83,22.00,281.39,256.10,43150.00,7.34,7.33,9.01,7.34,5.67,39.04
949,7.13,7.23,6.63,6.45,90984.48,64912.64,43207.95,1245659.37,3204.52,330653.02,835483.64,584.25,391823.23,317566.14,54.48,28.73,33675.47,149.85,114.82,0.50,25.74,289.56,239.13,46780.00,7.28,7.28,8.93,7.28,5.62,38.65
950,7.16,7.21,6.74,6.41,80009.53,64651.01,26144.95,1250020.39,3194.80,316229.80,938611.48,585.67,361613.62,267757.27,45.10,27.52,32582.48,139.01,111.66,0.81,11.97,284.17,214.20,42310.00,7.21,7.22,8.82,7.21,5.59,38.41
951,7.17,7.17,6.93,6.40,62592.86,64236.66,21591.19,1196746.13,3173.55,307563.31,920489.35,590.06,295617.34,282348.52,38.46,25.04,32816.22,137.59,117.08,1.60,14.69,261.59,228.93,36640.00,7.14,7.17,8.72,7.14,5.56,38.31
952,7.14,7.01,6.89,6.39,80792.92,64352.09,35184.63,1610455.73,3150.45,338416.28,1097848.91,591.16,285045.32,312367.76,48.06,31.95,34949.86,163.30,116.93,0.54,20.12,325.11,257.11,41120.00,7.07,7.12,8.59,7.07,5.55,38.21
953,7.18,6.92,6.85,6.40,90671.52,64129.43,36659.87,1455949.81,3134.99,378833.43,998980.77,591.61,347643.36,426007.57,53.33,30.77,42582.23,153.54,112.85,0.68,21.86,322.16,227.17,45240.00,6.99,7.07,8.40,6.99,5.58,38.35
954,7.20,6.92,6.82,6.40,91023.51,64057.61,36861.15,1338276.42,3178.40,397898.59,1118851.18,588.61,387304.48,475854.14,45.49,28.12,38786.07,170.08,118.32,0.49,24.47,315.07,256.25,40860.00,6.91,7.03,8.11,6.91,5.70,38.38
955,7.20,6.95,6.73,6.41,89472.14,63939.16,37351.19,1405394.00,3189.53,394955.00,978937.03,587.86,406616.90,386083.83,28.19,29.50,33366.09,156.47,110.96,0.38,22.27,302.95,230.01,38570.00,6.83,6.99,7.85,6.83,5.81,38.52


In [18]:
predictions_next_day.to_csv('skforecast/predicciones.csv', index=False)

Si predice maso menos bien para estas variables hacer lo siguiente:
- tengo forma de tunear la libreria para que performe mejor?
- existe alguna otra libreria de auto ml pra series que funcione mejor? 


- Armar el clasificador (LSTM o CONV), entrenandolo sobre los datos del dataset original ( sin los ultimos 5 registros)
- Probar como predice el target (Tendencia) para los ultimos 5 registros con los datos generados por los modelos